In [ ]:
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Validation_Input.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Validation_GroundTruth.zip
# !wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Test_Input.zip

--2022-12-19 15:57:30--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 52.217.86.92, 52.217.138.129, 52.217.72.204, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|52.217.86.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11165358566 (10G) [application/zip]
Saving to: ‘ISIC2018_Task1-2_Training_Input.zip’

ISIC2018_Task1-2_Tr 100%[===================>]  10.40G  66.1MB/s    in 2m 17s  

2022-12-19 15:59:47 (77.7 MB/s) - ‘ISIC2018_Task1-2_Training_Input.zip’ saved [11165358566/11165358566]

--2022-12-19 15:59:47--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 52.217.104.20, 52.217.138.33, 52.217.198.89, ...
Connecting to isic-challenge-data.s3.amazonaws.co

In [ ]:
!unzip ./ISIC2018_Task1-2_Training_Input.zip
!rm ./ISIC2018_Task1-2_Training_Input.zip
!unzip ./ISIC2018_Task1_Training_GroundTruth.zip
!rm ./ISIC2018_Task1_Training_GroundTruth.zip
!unzip ./ISIC2018_Task1-2_Validation_Input.zip
!rm ./ISIC2018_Task1-2_Validation_Input.zip
!unzip ./ISIC2018_Task1_Validation_GroundTruth.zip
!rm ./ISIC2018_Task1_Validation_GroundTruth.zip
!mkdir ./Preproc

Streaming output truncated to the last 5000 lines.
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000493.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000494.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000495.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000496.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000498.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000499.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000500.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000501.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000503.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000504.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000505.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000506.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000507.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_0000508.jpg  
  inflating: ISIC2018_Task1-2_Training_Input/ISIC_000

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
import os
import torch
from torch import nn as nn
# import cv2 as cv
import torchvision
from tqdm import tqdm
from matplotlib import pyplot as plt
import torchinfo
from DoubleUNet import DoubleUNet

In [3]:
TRAIN_INPUT_DIR = './ISIC2018_Task1-2_Training_Input/'
TRAIN_GT_DIR = './ISIC2018_Task1_Training_GroundTruth/'

VAL_INPUT_DIR = './ISIC2018_Task1-2_Validation_Input/'
VAL_GT_DIR = './ISIC2018_Task1_Validation_GroundTruth/'

BATCH_SIZE = 1
LEARNING_RATE = 1e-4
EPOCHS = 100

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Device: {DEVICE}")

Device: cuda


In [4]:
training_input_files = os.listdir(TRAIN_INPUT_DIR)
for filename in training_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    training_input_files.remove(filename)
    print(filename)
n_train = len(training_input_files)

val_input_files = os.listdir(VAL_INPUT_DIR)
for filename in val_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    val_input_files.remove(filename)
    print(filename)
n_val = len(val_input_files)
n_train -= n_train % BATCH_SIZE
n_val -= n_val % BATCH_SIZE
print(n_train, n_val)

ATTRIBUTION.txt
LICENSE.txt
ATTRIBUTION.txt
LICENSE.txt
2594 100


In [4]:
train_X = torch.zeros((BATCH_SIZE, 3, 256, 384)).to(DEVICE)
train_y = torch.zeros((BATCH_SIZE, 1, 256, 384)).to(DEVICE)
nb = n_train//BATCH_SIZE
with tqdm(total=n_train) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = training_input_files[i]
            in_image = torchvision.io.read_image(TRAIN_INPUT_DIR + filename).to(DEVICE)
            gt_image = torchvision.io.read_image(TRAIN_GT_DIR + filename[:-4] + "_segmentation.png").to(DEVICE)
            train_X[i-l, :, :, :] = torchvision.transforms.functional.resize(in_image, [256, 384])
            train_y[i-l, :, :, :] = torchvision.transforms.functional.resize(gt_image, [256, 384])
            in_image, gt_image = None, None
            torch.cuda.empty_cache()
            pbar.update(1)

        train_y = (train_y//128)
        torch.save(train_X, f"./Preproc/train_X_{f}.torch")
        torch.save(train_y, f"./Preproc/train_y_{f}.torch")
train_X, train_y = None, None
torch.cuda.empty_cache()
   
val_X = torch.zeros((BATCH_SIZE, 3, 256, 384)).to(DEVICE)
val_y = torch.zeros((BATCH_SIZE, 1, 256, 384)).to(DEVICE)
nb = n_val//BATCH_SIZE
with tqdm(total=n_val) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = val_input_files[i]
            in_image = torchvision.io.read_image(VAL_INPUT_DIR + filename).to(DEVICE)
            gt_image = torchvision.io.read_image(VAL_GT_DIR + filename[:-4] + "_segmentation.png").to(DEVICE)
            val_X[i-l, :, :, :] = torchvision.transforms.functional.resize(in_image, [256, 384])
            val_y[i-l, :, :, :] = torchvision.transforms.functional.resize(gt_image, [256, 384])
            in_image, gt_image = None, None
            torch.cuda.empty_cache()
            pbar.update(1)

        val_y = (val_y//128)
        torch.save(val_X, f"./Preproc/val_X_{f}.torch")
        torch.save(val_y, f"./Preproc/val_y_{f}.torch")
val_X, val_y = None, None
torch.cuda.empty_cache()
   

  0%|                                                                                 | 1/2594 [00:01<52:11,  1.21s/it]C:\Users\Izadi\AppData\Local\Temp\ipykernel_8068\1583826877.py:18: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  train_y = (train_y//128)
  1%|▊                                                                                 | 1/100 [00:00<00:37,  2.67it/s]C:\Users\Izadi\AppData\Local\Temp\ipykernel_8068\1583826877.py:41: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative value

In [ ]:
def boxfilter(imgSrc, r):
  h, w = imgSrc.shape[2:]
  imgDst = torch.zeros(imgSrc.shape)
  imgCum = torch.cumsum(imgSrc, 2)
  
  imgDst[:,:,:r,:] = imgCum[:,:,r:r*2,:]
  imgDst[:,:,r:h-r,:] = imgCum[:,:,2*r:h,:] - imgCum[:,:,:h-2*r,:]
  imgDst[:,:,h-r:,:] = imgCum[:,:,h-1:,:].repeat((1, 1, r, 1)) - imgCum[:,:,h-2*r:h-r,:]

  imgCum = torch.cumsum(imgDst, 3)
  imgDst[:,:,:,:r] = imgCum[:,:,:,r:r*2]
  imgDst[:,:,:,r:w-r] = imgCum[:,:,:,2*r:w] - imgCum[:,:,:,:w-2*r]
  imgDst[:,:,:,w-r:] = imgCum[:,:,:,w-1:].repeat((1, 1, 1, r)) - imgCum[:,:,:,w-2*r:w-r]
  return imgDst

In [5]:
def train(model):
  num_batches = n_train // BATCH_SIZE
  model.train()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    for batch in range(num_batches):
      X = torch.load(f"./Preproc/train_X_{batch}.torch")
      y = torch.load(f"./Preproc/train_y_{batch}.torch")
      loss, met = model.fit(X, y)
      t_loss += loss*len(X)
      t_met += met
      proc += len(X)
      pbar.update(1)
      pbar.set_description(f"Avg. Loss: {t_loss/proc: .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_train, t_met/n_train)

def test(model):
  num_batches = n_val // BATCH_SIZE
  model.eval()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    with torch.no_grad():
      for batch in range(num_batches):
        X = torch.load(f"./Preproc/val_X_{batch}.torch")
        y = torch.load(f"./Preproc/val_y_{batch}.torch")
        loss, met = model.test(X, y)
        t_loss += loss*len(X)
        t_met += met
        proc += len(X)
        pbar.update(1)
        pbar.set_description(f"Avg. Loss: {t_loss/proc : .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_val, t_met/n_val)

In [ ]:
def encblock(in_features, out_features):
    block = nn.Sequential(
        nn.Conv2d(in_features, out_features, (3, 3), padding="same"),
        nn.BatchNorm2d(out_features),
        nn.ReLU(),
        nn.Conv2d(out_features, out_features, (1, 1), padding="same"),
        nn.BatchNorm2d(out_features),
        nn.ReLU(),
        nn.MaxPool2d((2, 2))
        )
    block = block.to(DEVICE)
    return block

In [ ]:
def decblock(in_features, out_features):
    block = nn.Sequential(
        nn.ConvTranspose2d(in_features, out_features, 2, stride=(2, 2)),
        nn.BatchNorm2d(out_features),
        nn.ReLU(),
        nn.Conv2d(out_features, out_features, (5, 5), padding="same"),
        nn.BatchNorm2d(out_features),
        nn.ReLU()
        )
    block = block.to(DEVICE)
    return block

In [15]:
class UNET(nn.Module):
  def __init__(self, loss_fn=None, optimizer=None, device=None):
    super(UNET, self).__init__()
    self.input = nn.Conv2d(3, 4, (1, 1),padding="same")
    
    self.enc1 = encblock(4, 8)
    self.enc2 = encblock(8, 16)
    self.enc3 = encblock(16, 32)
    self.enc4 = encblock(32, 64)
    
    self.dec4 = decblock(64, 32)
    self.dec3 = decblock(64, 16)
    self.dec2 = decblock(32, 8)
    self.dec1 = decblock(16, 4)
    
    self.output = nn.Conv2d(8, 1, (1, 1), padding="same")

    if (loss_fn is None):
      self.loss_fn = nn.BCELoss()
    else:
      self.loss_fn = loss_fn
    
    if (optimizer is None):
      self.optimizer = torch.optim.Adam(self.parameters(), lr=LEARNING_RATE)
    else:
      self.optimizer = optimizer(self.parameters(), lr=LEARNING_RATE)
    
    if (device is None):
      self.device = "cuda" if torch.cuda.is_available() else "cpu"
    else:
      self.device = device
    
    self = self.to(self.device)


  def forward(self, X):
      X = self.input(X)
      h1 = self.enc1(X)
      h2 = self.enc2(h1)
      h3 = self.enc3(h2)
      h4 = self.enc4(h3)
      h4 = self.dec4(h4)
      h3 = torch.cat((h3, h4), dim=1)
      h4 = None
      h3 = self.dec3(h3)
      h2 = torch.cat((h2, h3), dim=1)
      h3 = None
      h2 = self.dec2(h2)
      h1 = torch.cat((h1, h2), dim=1)
      h2 = None
      h1 = self.dec1(h1)
      X = torch.cat((X, h1), dim=1)
      h1 = None
      X = self.output(X)
      X = torch.sigmoid(X)
      return X
  
  
  def fit(self, X, y):
      X, y = X.to(self.device), y.to(self.device)
      torch.cuda.empty_cache()
      pred = self(X)
      self.optimizer.zero_grad()
      loss = self.loss_fn(pred, y)
      loss.backward()
      self.optimizer.step()
      loss = loss.item()
      y = (y > 0.5)
      pred = (pred > 0.5)
      numt = torch.numel(y[0])
      TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
      TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
      FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
      FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
      return (loss, torch.tensor([TP, FP, FN, TN]))

  def test(self, X, y):
      X, y = X.to(self.device), y.to(self.device)
      pred = self(X)
      loss = self.loss_fn(pred, y).item()
      y = (y > 0.5)
      pred = (pred > 0.5)
      numt = torch.numel(y[0])
      TP = torch.sum(torch.bitwise_and(y, pred)).item() / numt
      TN = torch.sum(torch.bitwise_not(torch.bitwise_or(y, pred))).item() / numt
      FN = torch.sum(torch.bitwise_and(y, torch.bitwise_not(pred))).item() / numt
      FP = torch.sum(torch.bitwise_and(torch.bitwise_not(y), pred)).item() / numt
      return (loss, torch.tensor([TP, FP, FN, TN]))

In [6]:
unet = DoubleUNet().to(DEVICE)

C:\Users\Izadi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\Izadi\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
A = unet(torch.randn((1, 3, 256, 384)).to(DEVICE))

In [12]:
A[1].shape

torch.Size([1, 1, 256, 384])

In [7]:
for t in range(EPOCHS):
  torch.cuda.empty_cache()
  print(f"Epoch {t+1} ---------------------")
  print(f"Training Set -----")
  train_loss, _ = train(unet)
  print(f"Validation Set -----")
  train_loss, met = test(unet)
  print(f"\rIoU: {met[0] / (met[0] + met[1] + met[2]): .4f}, Dice: {met[0] / (met[0] + 0.5 * (met[1] + met[2])): .4f}, Acc: {met[0] + met[3] : .4f}")

Epoch 1 ---------------------
Training Set -----


Avg.Loss: 0.0000, Avg. Accuracy: 0.0000:   0%|                                                | 0/2594 [00:04<?, ?it/s]


RuntimeError: Unable to find a valid cuDNN algorithm to run convolution

In [ ]:
X = torch.load(f"./Preproc/train_X_0.torch")
y = torch.load(f"./Preproc/train_y_0.torch")
loss  = unet.fit(X, y)

In [ ]:
unet.eval()
with torch.no_grad():
  pred = unet(X)

In [ ]:
plt.figure()
plt.imshow((pred[3,0].cpu().detach() > 0.5))
plt.figure()
plt.imshow(y[3,0].cpu())

In [ ]:
X, y = X.to(self.device), y.to(self.device)
      torch.cuda.empty_cache()
      pred = self(X)
      X = None
      torch.cuda.empty_cache()
      self.optimizer.zero_grad()
      loss = self.loss_fn(pred, y)
      y = None
      torch.cuda.empty_cache()
      loss.backward()
      self.optimizer.step()
      torch.cuda.empty_cache()
      return loss.item()